In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_iNitroY_Classification_DLNN_CORENup_v2"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 16
shuffle = True
seed = None

input_data_folder = "Data\\iNitroY-Deep-Dataset"
pos_data_file = "raw-nitrotyrosine-pos.fasta"
neg_data_file = "cdhit70-nitrotyr-neg.fasta"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score

import math

from Bio import SeqIO

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_nt(sequence, char_dict):
    
    seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
    i = 0
    for single_character in sequence:
        if(single_character.upper() in char_dict.keys()):
            seq_encoded[i][char_dict[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in NT sequence: '+sequence)
    return seq_encoded

In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [7]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_CORENup(input_seq_shape = (41, 21),
                 conv_filters_per_layer_1 = 10, kernel_length_1 = 10, conv_strides_1 = 1, ## 1st Convolutional layer parameters
                 max_pool_width_1 = 3, max_pool_stride_1 = 3, ## 1st Maxpool layer parameters
                 lstm_decode_units = 10, ## LSTM layer parameters
                 conv_filters_per_layer_2 = 10,  kernel_length_2 = 5, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
                 max_pool_width_2 = 3, max_pool_stride_2 = 3, ## 2nd Maxpool layer parameters
                 dense_decode_units = 32, ## Dense layer parameters
                 prob = 0.5, learn_rate = 0.0005, 
                 loss = 'binary_crossentropy', metrics = None):
    
    beta = 0.001
    
    ######################################################################################################
    ########  SEQUENCE  ##################################################################################
    ######################################################################################################
    
    input1 = tf.keras.layers.Input(shape=input_seq_shape)

    x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
                                strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta), 
                                padding = "same")(input1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
    x1 = tf.keras.layers.Dropout(prob)(x1)

    ## LSTM Path

    x2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    
    x2 = tf.keras.layers.Dropout(prob)(x2)
    
    x2 = tf.keras.layers.Flatten()(x2)

    ## Conv Path

    x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
                                kernel_regularizer = tf.keras.regularizers.l2(beta), padding = 'same')(x1)
    x3 = tf.keras.layers.Activation('relu')(x3)
    x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
    x3 = tf.keras.layers.Dropout(prob)(x3)
    
    x3 = tf.keras.layers.Flatten()(x3)
    
    x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(x4)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

    return model

In [8]:
# for step in range(10):
#     initial_learning_rate=1e-1
#     decay_steps=10000
#     decay_rate=0.9
#     print(step, ':', initial_learning_rate * decay_rate ** (step / decay_steps))

In [9]:
DLNN_CORENup().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 41, 21)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 41, 10)       2110        ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 41, 10)       0           ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 13, 10)       0           ['activation[0][0]']             
                                                                                              

C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Prepare Dataset

In [10]:
def read_fasta_file(file_path):
    
    openFile = open(file_path)
    fastaSequences = SeqIO.parse(openFile, "fasta")

    name_list = []
    seq_list = []

    for fasta in fastaSequences: 
        name_list.append(fasta.id)
        seq_list.append(str(fasta.seq))

    openFile.close()
    
    return name_list, seq_list

In [11]:
##################################################################################
##### read positive and negative files
##################################################################################

pos_file_path = os.path.join(input_data_folder, pos_data_file)
_, pos_seq_list = read_fasta_file(pos_file_path)

neg_file_path = os.path.join(input_data_folder, neg_data_file)
_, neg_seq_list = read_fasta_file(neg_file_path)

pos_seq_list = [val.replace('X', '-') for val in pos_seq_list]
neg_seq_list = [val.replace('X', '-') for val in neg_seq_list]

# remove duplicates in data
pos_seq_list = list(set(pos_seq_list))
neg_seq_list = list(set(neg_seq_list))

all_seq_list = pos_seq_list + neg_seq_list

all_seq_label_list = ([1] * len(pos_seq_list)) + ([0] * len(neg_seq_list))

##################################################################################
##### Create dictionary of all characters in the NT sequence 
##################################################################################
all_char_set = set({})
for val in [set(val) for val in all_seq_list]:
    all_char_set = all_char_set.union(val)
all_char_list = list(all_char_set)
all_char_list.sort()
all_char_dict = {}
for i in range(len(all_char_list)):
    all_char_dict[all_char_list[i]] = i
    
##################################################################################
##### Create OHE of all sequences
##################################################################################
all_seq_OHE_list = [one_hot_encode_nt(val, all_char_dict)
                    for val in all_seq_list]

##################################################################################
##### Create numpy array of features and sequences
##################################################################################

## create the features and labels datasets for the training
features = np.array(all_seq_OHE_list)
labels = np.array(all_seq_label_list)
labels = labels.reshape((labels.shape[0], 1))

##################################################################################
##### Divide into Train/Independent datasets
##################################################################################

train_features, indpe_features, train_labels, indpe_labels = train_test_split(features, labels, 
                                                                              stratify=labels, test_size=0.3, 
                                                                              random_state=seed, shuffle=shuffle)

##################################################################################
##### Generate Folds from training dataset, and store to file
##################################################################################

## Generate the k-fold dataset
folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

## Write the independent test dataset to file
pickle.dump([indpe_features, indpe_labels], open(os.path.join(foldPath, 'independent_dataset.pickle'), "wb"))

##################################################################################

input_seq_shape = features[0].shape

# Training

In [12]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)


Train/Test model on Fold #0.
Epoch 1/100


C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


24/24 [==============================] - ETA: 0s - loss: 0.7798
Epoch 1: val_loss improved from inf to 0.71121, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 5s 31ms/step - loss: 0.7798 - val_loss: 0.7112
Epoch 2/100
18/24 [=====================>........] - ETA: 0s - loss: 0.7394
Epoch 2: val_loss improved from 0.71121 to 0.67248, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 0s 10ms/step - loss: 0.7354 - val_loss: 0.6725
Epoch 3/100
17/24 [====================>.........] - ETA: 0s - loss: 0.6854
Epoch 3: val_loss improved from 0.67248 to 0.65439, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 0s 10ms/step - loss: 0.6897 - val_loss: 0.6544
Epoch 4/100
17/24 [====================>.........] - 

24/24 [==============================] - ETA: 0s - loss: 0.2451
Epoch 55: val_loss improved from 0.24087 to 0.23567, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 0s 11ms/step - loss: 0.2451 - val_loss: 0.2357
Epoch 56/100
24/24 [==============================] - ETA: 0s - loss: 0.2664
Epoch 56: val_loss improved from 0.23567 to 0.23192, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 0s 11ms/step - loss: 0.2664 - val_loss: 0.2319
Epoch 57/100
24/24 [==============================] - ETA: 0s - loss: 0.2423
Epoch 57: val_loss improved from 0.23192 to 0.22291, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 0s 11ms/step - loss: 0.2423 - val_loss: 0.2229
Epoch 58/100
17/24 [====================>...

Epoch 88/100
17/24 [====================>.........] - ETA: 0s - loss: 0.1418
Epoch 88: val_loss did not improve from 0.18689
24/24 [==============================] - 0s 8ms/step - loss: 0.1572 - val_loss: 0.1918
Epoch 89/100
17/24 [====================>.........] - ETA: 0s - loss: 0.1439
Epoch 89: val_loss did not improve from 0.18689
24/24 [==============================] - 0s 9ms/step - loss: 0.1278 - val_loss: 0.2050
Epoch 90/100
22/24 [==========================>...] - ETA: 0s - loss: 0.1439
Epoch 90: val_loss did not improve from 0.18689
24/24 [==============================] - 0s 9ms/step - loss: 0.1440 - val_loss: 0.1882
Epoch 91/100
24/24 [==============================] - ETA: 0s - loss: 0.1488
Epoch 91: val_loss did not improve from 0.18689
24/24 [==============================] - 0s 8ms/step - loss: 0.1488 - val_loss: 0.1922
Epoch 92/100
17/24 [====================>.........] - ETA: 0s - loss: 0.1569
Epoch 92: val_loss improved from 0.18689 to 0.18351, saving model to Result

...

C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


24/24 [==============================] - ETA: 0s - loss: 0.8317
Epoch 1: val_loss improved from inf to 0.68672, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
24/24 [==============================] - 2s 26ms/step - loss: 0.8317 - val_loss: 0.6867
Epoch 2/100
24/24 [==============================] - ETA: 0s - loss: 0.7261
Epoch 2: val_loss improved from 0.68672 to 0.65247, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
24/24 [==============================] - 0s 11ms/step - loss: 0.7261 - val_loss: 0.6525
Epoch 3/100
24/24 [==============================] - ETA: 0s - loss: 0.6774
Epoch 3: val_loss improved from 0.65247 to 0.64416, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
24/24 [==============================] - 0s 10ms/step - loss: 0.6774 - val_loss: 0.6442
Epoch 4/100
17/24 [====================>.........] - 

24/24 [==============================] - 0s 10ms/step - loss: 0.5787 - val_loss: 0.5726
Epoch 27/100
17/24 [====================>.........] - ETA: 0s - loss: 0.5860
Epoch 27: val_loss improved from 0.57258 to 0.56535, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
24/24 [==============================] - 0s 10ms/step - loss: 0.5477 - val_loss: 0.5653
Epoch 28/100
17/24 [====================>.........] - ETA: 0s - loss: 0.5335
Epoch 28: val_loss improved from 0.56535 to 0.55661, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
24/24 [==============================] - 0s 11ms/step - loss: 0.5367 - val_loss: 0.5566
Epoch 29/100
23/24 [===========================>..] - ETA: 0s - loss: 0.5266
Epoch 29: val_loss improved from 0.55661 to 0.55228, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
24/24 [========================

Epoch 59/100
17/24 [====================>.........] - ETA: 0s - loss: 0.1835
Epoch 59: val_loss did not improve from 0.43843
24/24 [==============================] - 0s 8ms/step - loss: 0.1962 - val_loss: 0.4793
Epoch 60/100
21/24 [=========================>....] - ETA: 0s - loss: 0.1716
Epoch 60: val_loss did not improve from 0.43843
24/24 [==============================] - 0s 10ms/step - loss: 0.1685 - val_loss: 0.5296
Epoch 61/100
24/24 [==============================] - ETA: 0s - loss: 0.1856
Epoch 61: val_loss did not improve from 0.43843
24/24 [==============================] - 0s 9ms/step - loss: 0.1856 - val_loss: 0.5093
Epoch 62/100
24/24 [==============================] - ETA: 0s - loss: 0.1599
Epoch 62: val_loss did not improve from 0.43843
24/24 [==============================] - 0s 9ms/step - loss: 0.1599 - val_loss: 0.5455
Epoch 63/100
24/24 [==============================] - ETA: 0s - loss: 0.1671
Epoch 63: val_loss did not improve from 0.43843
24/24 [===================

Epoch 98/100
24/24 [==============================] - ETA: 0s - loss: 0.1095
Epoch 98: val_loss did not improve from 0.43843
24/24 [==============================] - 0s 9ms/step - loss: 0.1095 - val_loss: 0.6432
Epoch 99/100
17/24 [====================>.........] - ETA: 0s - loss: 0.1258
Epoch 99: val_loss did not improve from 0.43843
24/24 [==============================] - 0s 8ms/step - loss: 0.1286 - val_loss: 0.5910
Epoch 100/100
17/24 [====================>.........] - ETA: 0s - loss: 0.0729
Epoch 100: val_loss did not improve from 0.43843
24/24 [==============================] - 0s 8ms/step - loss: 0.0896 - val_loss: 0.6293


## k-fold Training evaluation

In [13]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.906428,0.862443,0.893417,0.919635,0.862443,0.739371
Train,0.984083,0.967855,0.995150,0.989553,0.967855,0.957221


In [14]:
# batch 16
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.943310	0.923092	0.965850	0.957041	0.923092	0.881812
# Train	0.997427	0.995646	0.999898	0.998589	0.995646	0.994615

In [15]:
evaluations_df[evaluations_df["Train_Test"] == "Test"]

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.978947,1.000000,"[0.0, 0.041666666666666664, 0.9166666666666666...","[0.0, 0.0, 0.0, 0.04225352112676056, 0.0422535...","[1.9875169, 0.9875168, 0.81538314, 0.08427684,...",0.975352,0.972603,1.000000,0.944221
3,1,Test,0.893617,0.782609,"[0.0, 0.0, 0.13043478260869565, 0.130434782608...","[0.0, 0.014084507042253521, 0.0140845070422535...","[1.9530256, 0.95302564, 0.91644675, 0.9053425,...",0.913656,0.929577,0.782609,0.712186
5,2,Test,0.893617,0.842105,"[0.0, 0.043478260869565216, 0.5217391304347826...","[0.0, 0.0, 0.0, 0.014084507042253521, 0.014084...","[1.9249774, 0.9249774, 0.80512834, 0.8016588, ...",0.889161,0.906667,0.842105,0.699462
7,3,Test,0.904255,0.937500,"[0.0, 0.0, 0.6521739130434783, 0.6521739130434...","[0.0, 0.014084507042253521, 0.0140845070422535...","[1.9252974, 0.9252974, 0.55983454, 0.30217847,...",0.831598,0.897436,0.937500,0.729907
9,4,Test,0.861702,0.750000,"[0.0, 0.043478260869565216, 0.1739130434782608...","[0.0, 0.0, 0.0, 0.014084507042253521, 0.014084...","[1.9508462, 0.95084625, 0.8793011, 0.8486935, ...",0.857318,0.891892,0.750000,0.611082


# Independent data

## Using k-fold Models

### Performance of each k-fold model

In [16]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.875248,0.809524,0.863026,0.893346,0.809524,0.64846


In [17]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.881188,0.809524,"[0.0, 0.02, 0.52, 0.52, 0.54, 0.54, 0.6, 0.6, ...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.9910834, 0.9910834, 0.9048365, 0.9032092, 0...",0.888289,0.900000,0.809524,0.667183
1,1,Independent,0.866337,0.810811,"[0.0, 0.02, 0.28, 0.28, 0.5, 0.5, 0.52, 0.52, ...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.969076, 0.96907604, 0.90845865, 0.90223265,...",0.863158,0.878788,0.810811,0.618063
2,2,Independent,0.886139,0.813953,"[0.0, 0.02, 0.22, 0.22, 0.56, 0.56, 0.58, 0.58...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.9743633, 0.97436327, 0.9182227, 0.9003077, ...",0.877500,0.905660,0.813953,0.682536
3,3,Independent,0.851485,0.833333,"[0.0, 0.02, 0.3, 0.3, 0.42, 0.42, 0.44, 0.44, ...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.9477913, 0.9477914, 0.7897756, 0.7895232, 0...",0.793026,0.854651,0.833333,0.566887
4,4,Independent,0.891089,0.780000,"[0.0, 0.02, 0.14, 0.14, 0.24, 0.24, 0.5, 0.5, ...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.9304771, 0.93047714, 0.9067157, 0.9054375, ...",0.893158,0.927632,0.780000,0.707632


### Mean score with k-fold models

In [18]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.930693,0.928571,0.882368,0.93125,0.928571,0.808511


### Voting score with k-fold models

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.905941,0.878049,0.884934,0.913043,0.878049,0.737268


## Using New Model

Train one model on full data from training. Predict and evaluate on Independent data.

In [20]:
model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/100


C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


30/30 [==============================] - ETA: 0s - loss: 0.7621
Epoch 1: val_loss improved from inf to 0.70310, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
30/30 [==============================] - 2s 29ms/step - loss: 0.7621 - val_loss: 0.7031
Epoch 2/100
23/30 [======================>.......] - ETA: 0s - loss: 0.6919
Epoch 2: val_loss improved from 0.70310 to 0.66304, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
30/30 [==============================] - 0s 10ms/step - loss: 0.6896 - val_loss: 0.6630
Epoch 3/100
25/30 [========================>.....] - ETA: 0s - loss: 0.6843
Epoch 3: val_loss improved from 0.66304 to 0.65584, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
30/30 [==============================] - 0s 10ms/step - loss: 0.6810 - val_loss: 0.6558
Epoch 4/100
25/30 [========================>.....] - ETA: 0s - loss:

Epoch 61/100
25/30 [========================>.....] - ETA: 0s - loss: 0.1679
Epoch 61: val_loss did not improve from 0.41055
30/30 [==============================] - 0s 9ms/step - loss: 0.1643 - val_loss: 0.4857
Epoch 62/100
25/30 [========================>.....] - ETA: 0s - loss: 0.1873
Epoch 62: val_loss did not improve from 0.41055
30/30 [==============================] - 0s 9ms/step - loss: 0.1841 - val_loss: 0.4917
Epoch 63/100
25/30 [========================>.....] - ETA: 0s - loss: 0.1855
Epoch 63: val_loss did not improve from 0.41055
30/30 [==============================] - 0s 9ms/step - loss: 0.1747 - val_loss: 0.5325
Epoch 64/100
25/30 [========================>.....] - ETA: 0s - loss: 0.1577
Epoch 64: val_loss did not improve from 0.41055
30/30 [==============================] - 0s 9ms/step - loss: 0.1560 - val_loss: 0.4987
Epoch 65/100
25/30 [========================>.....] - ETA: 0s - loss: 0.1404
Epoch 65: val_loss did not improve from 0.41055
30/30 [====================

Epoch 100/100
25/30 [========================>.....] - ETA: 0s - loss: 0.1098
Epoch 100: val_loss did not improve from 0.41055
30/30 [==============================] - 0s 9ms/step - loss: 0.1180 - val_loss: 0.7192


In [21]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.881188,0.795455,0.867763,0.905063,0.795455,0.669988


In [23]:
# out_fasta_file_name = '.'.join((training_data_file.split('.')[0], 'fasta'))
# out_fasta_file_path = os.path.join(input_data_folder, out_fasta_file_name)

# count = 0
# list_seqs = list(train_data['Sequence'])

# with open(out_fasta_file_path, "w") as out_file_obj:
#     for strLine in list_seqs:
        
#         #Output the header
#         out_file_obj.write(">" + str(count+1) + "\n")
#         out_file_obj.write(strLine + "\n")
        
#         count += 1